# 제주 한달살기 숙소 추천을 위한 분석 - 전처리


### 미스터멘션 - https://www.mrmention.co.kr/
### 한달살기 전문사이트
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- case 155건
- 주소 처리하고, 부족한 주소는 수작업으로 입력


In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
df_base = pd.read_excel('./data/mrmention_addr_20240118.xlsx')
df_base.head(3)

,Unnamed: 0,title,day,price,rating,reviewNum,location,검색어,주소1,주소2
0,0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,29박,1193060,4.5,82,안덕면,인제주,none,none
1,1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,제주 제주시 애월읍 고성남서길 31-6
2,2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,제주 제주시 원노형3길 3 1층


In [3]:
# 필요없는 columns 제거
df_base = df_base.iloc[:, 1:-1]
df_base.head(3)

,title,day,price,rating,reviewNum,location,검색어,주소1
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,29박,1193060,4.5,82,안덕면,인제주,none
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층


In [4]:
# 컬럼명 변경
df_base.rename(columns= {'검색어' : 'pension_keyword', '주소1' : 'addr'}, inplace= True)
df_base.head(3)

,title,day,price,rating,reviewNum,location,pension_keyword,addr
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,29박,1193060,4.5,82,안덕면,인제주,none
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층


In [5]:
# 주소가 확인되는 숙소 선택
df_base = df_base[df_base['addr'] != 'none']
print(len(df_base))

94


In [6]:
# 이름이 중복되는 컬럼 존재
df_base.duplicated('title').sum()

3

In [7]:
df_base[df_base.duplicated('title')]

,title,day,price,rating,reviewNum,location,pension_keyword,addr
62,[제주/구좌] 덕천씨엘빌,29박,2871000,4.9,11,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
63,[제주/구좌] 덕천씨엘빌,29박,2871000,4.7,9,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
64,[제주/구좌] 덕천씨엘빌,29박,2871000,4.7,9,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101


In [8]:
# '[제주/구좌] 덕천씨엘빌'가 4개로 등록되어서 하나로 바꾼다.
df_base[df_base['title'] == '[제주/구좌] 덕천씨엘빌'] 

,title,day,price,rating,reviewNum,location,pension_keyword,addr
61,[제주/구좌] 덕천씨엘빌,29박,2871000,4.9,11,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
62,[제주/구좌] 덕천씨엘빌,29박,2871000,4.9,11,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
63,[제주/구좌] 덕천씨엘빌,29박,2871000,4.7,9,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
64,[제주/구좌] 덕천씨엘빌,29박,2871000,4.7,9,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101


In [9]:
# 두개의 정보를 하나로 합치기
new_rating = round(((4.9 * 11) + (4.7 * 9)) / 20, 1)
temp = pd.DataFrame([['[제주/구좌] 덕천씨엘빌', '29박', 2871000, new_rating, 20, '구좌읍', '[제주/구좌]덕천씨엘빌', '제주 제주시 구좌읍 덕평로 121-101']])
temp.columns = ['title', 'day','price', 'rating', 'reviewNum', 'location', 'pension_keyword', 'addr']
temp

,title,day,price,rating,reviewNum,location,pension_keyword,addr
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101


In [10]:
# 총 91개의 케이스 존재
temp2 = df_base[df_base['title'] != '[제주/구좌] 덕천씨엘빌'] 
df_base = pd.concat([temp, temp2])
df_base.reset_index(drop = True, inplace= True)
df_base

,title,day,price,rating,reviewNum,location,pension_keyword,addr
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층
3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1
4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동
...,...,...,...,...,...,...,...,...
86,[제주/구좌]세화해변에 위치한 감성 북스테이 #제주 워케이션 #한달살이 #book bar,29박,1505680,0.0,0,구좌읍,#bookbar,제주특별자치도 제주시 구좌읍 한동북1길 34-6
87,"[서귀포/성산] 청정 제주 동부를 정원에 한 가득 담은 낭만 명소, kia ora",29박,1221770,5.0,2,성산읍,kiaora,제주특별자치도 서귀포시 성산읍 난고로42번길 3
88,[제주시] 신축풀옵숙소 한달살기 '라온스테이',29박,1116500,4.5,10,공설로5길,한달살기'라온스테이',제주 서귀포시 성산읍 온평서로 28-17
89,[제주/애월] 유티에스빌리지,29박,1229872,4.0,45,애월읍,[제주/애월]유티에스빌리지,제주 제주시 애월읍 광령평화2길 170-2


In [11]:
df_base.to_csv('./data/mmention_base_20240118.csv', index=False)

### review데이터의 ammen (시설정보) 추가

In [12]:
df_detail =pd.read_csv('./data/mmention_review_20240118.csv')
df_detail.head(3)

,title,review,review_date,star,ammen
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,내 집처럼 쾌적하고 편안한 숙소에서 한달살이 잘 쉬고 갑니다. 조용하고 아담한 동네...,2023년 12월,5.0,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
1,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,제주는 단기여행으로 많이왔지만 한달살이는첨이라 많이설레고숙소도 기대 대고 정말편히잘...,2023년 11월,5.0,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
2,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,숙소가 생각보다 좋아서 일주일간 잘 지내다 갑니다. 내집 같은 느낌이었어요. 아이들...,2023년 10월,5.0,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...


In [13]:
df_ammen = df_detail[['title', 'ammen']]
df_ammen.head(3)

,title,ammen
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
1,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
2,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...


In [14]:
df_ammen.drop_duplicates(inplace= True)
df_ammen

C:\Users\dotho\AppData\Local\Temp\ipykernel_1064\1765370498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ammen.drop_duplicates(inplace= True)


,title,ammen
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
87,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,['주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 ...
111,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 헤어...
129,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면...
375,[제주/제주시] [한화리조트 X 미스터멘션] 제주 한화리조트,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 도어락 세면도구 구급 상자 ...
...,...,...
3589,[친환경 숙소] [제주/한림] #단독 독채 #테라스 #해먹 #논밭뷰 #옥상단상 '설레임',['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 수건'...
3601,[서귀포시] 제주화펜션,['주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 건조기 도어락 세...
3602,[제주/제주시] 테리의 코지하우스 #공항10분거리,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 엘리베이터 도어락 헤어...
3603,[제주/애월] 단체 가능한 조용하고 한적한 '복층 독채',"['주차장 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 헤어 드라이기', '전..."


In [15]:
# 두 데이터 프레임 합치기

In [16]:
df = pd.merge(df_base, df_ammen, how='left', on = 'title')
df

,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 ...
1,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 헤어 드...
2,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,['주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 ...
3,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면...
4,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"['주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용)', '전..."
...,...,...,...,...,...,...,...,...,...
87,[제주/구좌]세화해변에 위치한 감성 북스테이 #제주 워케이션 #한달살이 #book bar,29박,1505680,0.0,0,구좌읍,#bookbar,제주특별자치도 제주시 구좌읍 한동북1길 34-6,['주차장 에어컨 와이파이 도어락 세면도구 구급 상자 수건 다리미 헤어 드라이기 T...
88,"[서귀포/성산] 청정 제주 동부를 정원에 한 가득 담은 낭만 명소, kia ora",29박,1221770,5.0,2,성산읍,kiaora,제주특별자치도 서귀포시 성산읍 난고로42번길 3,['주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기...
89,[제주시] 신축풀옵숙소 한달살기 '라온스테이',29박,1116500,4.5,10,공설로5길,한달살기'라온스테이',제주 서귀포시 성산읍 온평서로 28-17,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 구급 ...
90,[제주/애월] 유티에스빌리지,29박,1229872,4.0,45,애월읍,[제주/애월]유티에스빌리지,제주 제주시 애월읍 광령평화2길 170-2,['주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁기 (...


In [17]:
df.drop_duplicates('title', inplace= True)
df.reset_index(drop=True, inplace= True)
df.head(5)

,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 ...
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,['주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 ...
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면...
3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"['주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용)', '전..."
4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급...


In [18]:
pensionID = [f'pension{x + 1}'  for x in range(len(df))]
df['pensionID'] = pensionID
df

,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 ...,pension1
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,['주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 ...,pension2
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면...,pension3
3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"['주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용)', '전...",pension4
4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동,['주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급...,pension5
...,...,...,...,...,...,...,...,...,...,...
86,[제주/구좌]세화해변에 위치한 감성 북스테이 #제주 워케이션 #한달살이 #book bar,29박,1505680,0.0,0,구좌읍,#bookbar,제주특별자치도 제주시 구좌읍 한동북1길 34-6,['주차장 에어컨 와이파이 도어락 세면도구 구급 상자 수건 다리미 헤어 드라이기 T...,pension87
87,"[서귀포/성산] 청정 제주 동부를 정원에 한 가득 담은 낭만 명소, kia ora",29박,1221770,5.0,2,성산읍,kiaora,제주특별자치도 서귀포시 성산읍 난고로42번길 3,['주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기...,pension88
88,[제주시] 신축풀옵숙소 한달살기 '라온스테이',29박,1116500,4.5,10,공설로5길,한달살기'라온스테이',제주 서귀포시 성산읍 온평서로 28-17,['주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 구급 ...,pension89
89,[제주/애월] 유티에스빌리지,29박,1229872,4.0,45,애월읍,[제주/애월]유티에스빌리지,제주 제주시 애월읍 광령평화2길 170-2,['주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁기 (...,pension90


In [19]:
df['ammen'] = df['ammen'].apply(lambda x: str(x).replace("'", "").replace("[", "").replace("]", ""))
df['ammen']




0     주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...
1     주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...
2     주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...
3     주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용), 전기포트 ...
4     주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...
                            ...                        
86    주차장 에어컨 와이파이 도어락 세면도구 구급 상자 수건 다리미 헤어 드라이기 TV ...
87    주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기 세...
88    주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 구급 상자...
89    주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁기 (공용...
90    주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기 세...
Name: ammen, Length: 91, dtype: object

In [20]:
df_temp = df['ammen'].str.split(",", expand=True)
df_temp.columns = ['ammen1', 'ammen2', 'ammen3', 'ammen4', 'ammen5']
df_temp.head(5)

,ammen1,ammen2,ammen3,ammen4,ammen5
0,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,바비큐 테라스,가족/ 어린이환영 취사가능 실내 금연,제주스러움 마당
1,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연,바다 근처 제주스러움 마당
2,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,전기포트 전자레인지 밥솥 냉장고 식기 (그릇),테라스,가족/ 어린이환영 취사가능 실내 금연,대중교통
3,주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용),전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연 반려동물 가능,제주스러움 마당
4,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,반려동물 키움 대중교통 바다 근처 제주스러움 휠체어 사용 가능


In [21]:
df = pd.concat([df, df_temp], axis=1)
df.head()

,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,pension1,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,바비큐 테라스,가족/ 어린이환영 취사가능 실내 금연,제주스러움 마당
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,pension2,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연,바다 근처 제주스러움 마당
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,pension3,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,전기포트 전자레인지 밥솥 냉장고 식기 (그릇),테라스,가족/ 어린이환영 취사가능 실내 금연,대중교통
3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용), 전기포트 ...",pension4,주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용),전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연 반려동물 가능,제주스러움 마당
4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,pension5,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,반려동물 키움 대중교통 바다 근처 제주스러움 휠체어 사용 가능


In [22]:
# 숙소주소 결합

In [23]:
df_addr = pd.read_excel('data/숙소주소.xlsx')
df_addr

,Unnamed: 0,검색어,주소1,주소2,숙소,위도,경도
0,0,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,제주 제주시 애월읍 고성남서길 31-6,펜션0,33.477466,126.381554
1,1,센터포인트N,제주 제주시 원노형3길 3 1층,제주 제주시 원노형3길 3 1층,펜션1,33.486360,126.485488
2,2,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1,제주도 제주시 애월읍 도치돌길 279-1,펜션2,33.410676,126.364335
3,4,제주한화리조트,제주 제주시 명림로 575-107,제주 제주시 명림로 575-107,펜션3,33.449003,126.637358
4,5,근처토비스콘도,제주 제주시 애월읍 애월로1길 8,제주 제주시 애월읍 애월로1길 8,펜션4,33.463077,126.312442
...,...,...,...,...,...,...,...
88,92,kiaora,제주특별자치도 서귀포시 성산읍 난고로42번길 3,제주특별자치도 서귀포시 성산읍 난고로42번길 3,펜션88,33.434302,126.904891
89,93,한달살기'라온스테이',제주 서귀포시 성산읍 온평서로 28-17,제주 서귀포시 성산읍 온평서로 28-17,펜션89,33.411588,126.897309
90,94,[제주/애월]유티에스빌리지,제주 제주시 애월읍 광령평화2길 170-2,제주 제주시 애월읍 광령평화2길 170-2,펜션90,33.430631,126.434387
91,95,신축펜션팀버하우스,제주 제주시 애월읍 곽지7길 11,none,펜션91,33.450931,126.307863


In [24]:
df_addr = df_addr[['검색어', '위도', '경도']]
df_addr

,검색어,위도,경도
0,'돌담하우스',33.477466,126.381554
1,센터포인트N,33.486360,126.485488
2,#제주스러움예예하우스,33.410676,126.364335
3,제주한화리조트,33.449003,126.637358
4,근처토비스콘도,33.463077,126.312442
...,...,...,...
88,kiaora,33.434302,126.904891
89,한달살기'라온스테이',33.411588,126.897309
90,[제주/애월]유티에스빌리지,33.430631,126.434387
91,신축펜션팀버하우스,33.450931,126.307863


In [25]:
df = df.merge(df_addr, how = 'left', left_on= 'pension_keyword', right_on='검색어')
df

,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5,검색어,위도,경도
0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,pension1,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,바비큐 테라스,가족/ 어린이환영 취사가능 실내 금연,제주스러움 마당,[제주/구좌]덕천씨엘빌,33.495082,126.753155
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,pension2,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연,바다 근처 제주스러움 마당,'돌담하우스',33.477466,126.381554
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,pension3,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,전기포트 전자레인지 밥솥 냉장고 식기 (그릇),테라스,가족/ 어린이환영 취사가능 실내 금연,대중교통,센터포인트N,33.486360,126.485488
3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용), 전기포트 ...",pension4,주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용),전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연 반려동물 가능,제주스러움 마당,#제주스러움예예하우스,33.410676,126.364335
4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,pension5,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,반려동물 키움 대중교통 바다 근처 제주스러움 휠체어 사용 가능,제주준이네웰스플레이스,33.256731,126.521220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,[제주/구좌]세화해변에 위치한 감성 북스테이 #제주 워케이션 #한달살이 #book bar,29박,1505680,0.0,0,구좌읍,#bookbar,제주특별자치도 제주시 구좌읍 한동북1길 34-6,주차장 에어컨 와이파이 도어락 세면도구 구급 상자 수건 다리미 헤어 드라이기 TV ...,pension87,주차장 에어컨 와이파이 도어락 세면도구 구급 상자 수건 다리미 헤어 드라이기 TV ...,전기포트 전자레인지 밥솥 부엌 (공용) 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연,아침식사 대중교통 바다 근처 제주스러움 관리비 없음 휠체어 사용 가능,#bookbar,33.541039,126.828865
87,"[서귀포/성산] 청정 제주 동부를 정원에 한 가득 담은 낭만 명소, kia ora",29박,1221770,5.0,2,성산읍,kiaora,제주특별자치도 서귀포시 성산읍 난고로42번길 3,주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기 세...,pension88,주차장 TV 케이블 TV 에어컨 와이파이 세면도구 구급 상자 수건 헤어 드라이기 세...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,바비큐 테라스,반려동물 가능 취사가능 실내 금연,바다 근처 제주스러움 마당,kiaora,33.434302,126.904891
88,[제주시] 신축풀옵숙소 한달살기 '라온스테이',29박,1116500,4.5,10,공설로5길,한달살기'라온스테이',제주 서귀포시 성산읍 온평서로 28-17,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 구급 상자...,pension89,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 구급 상자...,전자레인지 밥솥 식기 (그릇) 냉장고 전기포트,,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,대중교통,한달살기'라온스테이',33.411588,126.897309
89,[제주/애월] 유티에스빌리지,29박,1229872,4.0,45,애월읍,[제주/애월]유티에스빌리지,제주 제주시 애월읍 광령평화2길 170-2,주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁기 (공용...,pension90,주차장 TV 케이블 TV 에어컨 와이파이 세면도구 수건 헤어 드라이기 세탁기 (공용),전기포트 밥솥 식기 (그릇) 냉장고,바비큐 테라스,반려동물 가능 취사가능 실내 금연,제주스러움 관리비 없음 마당,[제주/애월]유티에스빌리지,33.430631,126.434387


In [26]:
df.drop(columns='검색어', inplace=True)
df.reset_index(inplace=True)
df.rename(columns= {'index' : 'id', '위도': 'latitude', '경도':'longitude'}, inplace= True)
df.head()

,id,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5,latitude,longitude
0,0,[제주/구좌] 덕천씨엘빌,29박,2871000,4.8,20,구좌읍,[제주/구좌]덕천씨엘빌,제주 제주시 구좌읍 덕평로 121-101,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,pension1,주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 세면도구 수건 다리미 헤어...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,바비큐 테라스,가족/ 어린이환영 취사가능 실내 금연,제주스러움 마당,33.495082,126.753155
1,1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍,'돌담하우스',제주 제주시 애월읍 구엄동3길 20,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,pension2,주차장 TV 케이블 TV 에어컨 와이파이 욕조 세탁기 (개인) 도어락 세면도구 수건...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연,바다 근처 제주스러움 마당,33.477466,126.381554
2,2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동,센터포인트N,제주 제주시 원노형3길 3 1층,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,pension3,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구...,전기포트 전자레인지 밥솥 냉장고 식기 (그릇),테라스,가족/ 어린이환영 취사가능 실내 금연,대중교통,33.486360,126.485488
3,3,[제주/애월]#숲속 #통나무집 #힐링스테이 #제주스러움 예예하우스,29박,970015,4.9,6,애월읍,#제주스러움예예하우스,제주 제주시 애월읍 도치돌길 279-1 제주도 제주시 애월읍 도치돌길 279-1,"주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용), 전기포트 ...",pension4,주차장 TV 케이블 TV 에어컨 와이파이 헤어 드라이기 세탁기 (공용),전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,테라스,가족/ 어린이환영 취사가능 실내 금연 반려동물 가능,제주스러움 마당,33.410676,126.364335
4,4,[서귀포시] #오션뷰 #풀옵션 #한달살기 제주준이네 웰스플레이스,29박,1722600,4.8,61,막동산로46번길,제주준이네웰스플레이스,제주 서귀포시 서호동,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,pension5,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 구급 상...,전기포트 전자레인지 밥솥 식기 (그릇) 냉장고,,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,반려동물 키움 대중교통 바다 근처 제주스러움 휠체어 사용 가능,33.256731,126.521220


In [27]:
df[df['ammen'] == 'nan']

,id,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5,latitude,longitude
22,22,"[제주시] 제주스카이라인 독채 휴양펜션 (단층독채, 잔디마당, 테라스)",29박,2663650,4.6,39,주르레남길,"잔디마당,테라스)",제주 제주시 주르레남길 19 제주스카이라인휴양펜션,nan,pension23,nan,None,None,None,None,33.441414,126.483190
65,65,[제주/조천] 10걸음만 나가면 바닷가 /풀옵션 건조기 몸만들어오세요~/ 인프라좋은...,29박,2392500,4.5,2,조천읍,제주를담은신촌블루스2호낚시가능,제주 제주시 조천읍 신촌북2길 36 1층,nan,pension66,nan,None,None,None,None,33.538006,126.620026


In [28]:
# 숙소시설 정보가 없는 경우 제외
df = df[df['ammen'] != 'nan']


In [29]:
df['ammen'].apply(lambda x: 1 if x.find('관리비 없음') >0 else 0).sum()


23

In [30]:
df[df['pensionID'] == 'pension80']

,id,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5,latitude,longitude
79,79,[제주/애월] 제주도 여행에 최적인 애월읍 숙소 아르본리조트,29박,550019,3.8,72,애월읍,숙소아르본리조트,제주 제주시 애월읍 평화로 2482,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 구급 상자 수건 헤어 드라이기 ...,pension80,주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 구급 상자 수건 헤어 드라이기 ...,냉장고 전자레인지 식기 (그릇) 전기포트 밥솥,수영장 바비큐,가족/ 어린이환영 반려동물 가능 취사가능 실내 금연,아침식사 반려동물 키움 대중교통 바다 근처 제주스러움 휠체어 사용 가능 마당 관리...,33.430999,126.419417


In [31]:
df['ammen'].apply(lambda x: 1 if x.find('대중교통') >0 else 0).sum()

60

In [32]:
df.to_csv('./data/fianl_data/pension.csv', index = False)

In [33]:
df[df['pensionID'] == 'pension23']

,id,title,day,price,rating,reviewNum,location,pension_keyword,addr,ammen,pensionID,ammen1,ammen2,ammen3,ammen4,ammen5,latitude,longitude


In [41]:
temp = df[df['ammen'].str.find('반려동물') > 0]

In [42]:
temp2 = temp[temp['ammen'].str.find('대중교통') > 0]

In [43]:
len(temp2)

18

In [55]:
score_list = temp2['reviewNum'].sort_values(ascending=False)
score1, score2, score3 = score_list.unique()[0:3].tolist()



In [56]:
score_list = temp2['reviewNum'].sort_values(ascending=False)
score1, score2, score3 = score_list.unique()[0:3].tolist()

164

In [69]:
temp2.__class__

pandas.core.frame.DataFrame